In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 3
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年02月04日"  # include
END_DATE = "2023年02月25日"  # include

TARGET_CATEGORY = "火焰雞VMAX"

"""
放逐_騎拉帝納VSTAR

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_白馬VMAX
阿爾_鋁鋼龍VMAX
純阿爾VSTAR

夢幻VMAX

密勒頓ex

洛奇亞VSTAR
一擊_洛奇亞

沙奈朵ex
沙奈朵_蒼響
沙奈朵_超夢V-UNION

起源帕路奇亞VSTAR

連擊武道熊師VMAX

達克萊伊VSTAR

洗翠索羅亞克VSTAR

--

飄香豚ex

雷吉鐸拉戈VSTAR

烈空坐VMAX

火焰雞VMAX



起源帝牙盧卡VSTAR	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1
"""

'\n放逐_騎拉帝納VSTAR\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_白馬VMAX\n阿爾_鋁鋼龍VMAX\n純阿爾VSTAR\n\n夢幻VMAX\n\n密勒頓ex\n\n洛奇亞VSTAR\n一擊_洛奇亞\n\n沙奈朵ex\n沙奈朵_蒼響\n沙奈朵_超夢V-UNION\n\n起源帕路奇亞VSTAR\n\n連擊武道熊師VMAX\n\n達克萊伊VSTAR\n\n洗翠索羅亞克VSTAR\n\n--\n\n飄香豚ex\n\n雷吉鐸拉戈VSTAR\n\n烈空坐VMAX\n\n火焰雞VMAX\n\n\n\n起源帝牙盧卡VSTAR\t1\n\n索羅風速狗\t1\n\n洛托姆VSTAR\t1\n\n月石\t1\n\n化石翼龍VSTAR\t1\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['夢幻VMAX', '交錯切換機', '化石翼龍VSTAR', 'LTB', 'LTB_勾魂眼_噴火龍', '雷吉艾勒奇VMAX', '密勒頓ex', 'Other_Lost', 'LTB_洗翠黏美龍VSTAR', '達克萊伊VSTAR', '起源帕路奇亞VSTAR', '白馬蕾冠王VMAX', '放逐_騎拉帝納VSTAR', '超夢 V-UNION', '阿爾宙斯VSTAR', '光輝無極汰那', '飛翔皮卡丘VMAX', '酋雷姆VMAX', '起源帝牙盧卡VSTAR', 'LTB_空之封印石_水超鋼雷', 'LTB_空之封印石_水超闘雷', 'LTB_蓋歐卡', 'LTB_空之封印石_other', 'LTB_空之封印石_超鋼雷', '阿羅拉六尾VSTAR', '阿爾_白馬VMAX', 'LTB_空之封印石_水超', '阿爾_飛天皮VMAX', '連擊武道熊師VMAX', 'LTB_空之封印石_超闘', 'LTB_空之封印石_水超鋼闘', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超雷', 'LTB_快龍V', 'LTB_空之封印石_悪水超鋼', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_草超鋼雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_超闘雷', 'LTB_空之封印石_草超鋼闘雷', 'LTB_空之封印石_超鋼闘雷', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超鋼', 'LTB_空之封印石_草超闘雷', 'LTB_空之封印石_水草超鋼雷', 'LTB_空之封印石_水草超鋼闘雷', 'LTB_空之封印石_悪水超闘', 'LTB_空之封印石_悪水超', '洛托姆VSTAR', 'カラマネロVMAX', 'LTB_古月鳥', '阿爾_騎拉帝納VSTAR', '雷吉鐸拉戈VSTAR', '洛奇亞VSTAR', '沙奈朵ex', '沙奈朵_超夢V-UNION', '沙奈朵_蒼響', '黑馬蕾冠王VMAX', 'Other_騎拉帝納VSTAR', '純阿爾VSTAR', '阿爾_鋁鋼龍VMAX', '烈空坐VMAX', '幻影索羅亞克', '藏飽栗鼠VMAX', '火焰雞VMAX', '洗翠索羅亞克VSTAR'

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.987022399902344e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_target_deck = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_target_deck = deck_cnt

In [11]:
deck_cnt_list = sorted(deck_cnt_list, reverse=True)
for num_deck, deck_name in deck_cnt_list:
    if num_deck > 0:
        print(f"{deck_name}\t{num_deck}\n")

放逐_騎拉帝納VSTAR	406

LTB	296

密勒頓ex	248

阿爾宙斯VSTAR	241

雷吉艾勒奇VMAX	237

交錯切換機	236

夢幻VMAX	195

洛奇亞VSTAR	189

一擊_洛奇亞	176

LTB_勾魂眼_噴火龍	119

Other_騎拉帝納VSTAR	118

飛翔皮卡丘VMAX	97

沙奈朵ex	90

阿爾_騎拉帝納VSTAR	79

連擊武道熊師VMAX	60

沙奈朵_蒼響	56

白馬蕾冠王VMAX	48

LTB_蓋歐卡	47

阿爾_飛天皮VMAX	45

起源帕路奇亞VSTAR	44

超夢 V-UNION	42

沙奈朵_超夢V-UNION	38

洗翠索羅亞克VSTAR	29

阿爾_鋁鋼龍VMAX	27

阿爾_白馬VMAX	22

黑馬蕾冠王VMAX	17

達克萊伊VSTAR	16

LTB_洗翠黏美龍VSTAR	16

純阿爾VSTAR	13

others	12

Other_Lost	12

光輝無極汰那	9

LTB_空之封印石_水超鋼闘	8

火焰雞VMAX	7

LTB_空之封印石_other	7

起源帝牙盧卡VSTAR	6

洛托姆VSTAR	6

LTB_空之封印石_水超闘雷	6

LTB_快龍V	6

飄香豚ex	5

烈空坐VMAX	5

酋雷姆VMAX	4

雷吉鐸拉戈VSTAR	3

藏飽栗鼠VMAX	3

阿羅拉六尾VSTAR	2

索羅風速狗	2

月石	2

幻影索羅亞克	2

LTB_空之封印石_水超鋼雷	2

LTB_空之封印石_水超	2

柱神	1

化石翼龍VSTAR	1

ヒスイ ダイケンキVSTAR	1

LTB_空之封印石_超闘	1

LTB_空之封印石_水超雷	1

LTB_空之封印石_水超闘	1

LTB_空之封印石_悪水超雷	1

LTB_空之封印石_悪水超闘	1

LTB_空之封印石_悪水超鋼	1

LTB_空之封印石_悪水超	1



In [12]:
# number of decks for the target category
print(number_target_deck)
if number_target_deck <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

7


In [13]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str < real_start_date or date_exclude_jp_str > real_end_date:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis="rows", numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis="rows")
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks

    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows - 1, num_rows - 2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [14]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{real_start_date}-{real_end_date}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.3f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if deck_id in ["avg_num_used", "pick_rate"]:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

/Users/blake.pan/anaconda3/envs/sandbox_crawler/lib/python3.7/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [15]:
df_list[0]

,date,prefecture,num_players,rank,火焰雞VMAX\nS8b 020/184,鐵炮魚\nS5R 022/070,章魚桶\nS5R 023/070,火焰雞V\nS8b 019/184,捷拉奧拉V\nS6K 024/070,帝王拿波V\nS5R 026/070,投擲猴\nS8b 088/184,連擊武道熊師VMAX\nS5R 051/070,恰雷姆V\nS7R 036/067,列陣兵\nS8b 204/184,連擊武道熊師V\nS8b 094/184,龍王蠍V\nS12a 085/172,捷拉奧拉VMAX\nSPZ 006/020,光輝甲賀忍蛙\nS12a 033/172,霓虹魚V\nSPD 002/020,仙子伊布V\nS6a 083/069,仙子伊布VMAX\nS6a 041/069,光輝無極汰那\nS12a 115/172,大尾狸\nS9 082/100,大牙狸\nS12a 121/172,野蠻鱸魚\nSI 105/414,捷拉奧拉V\nS6K 073/070
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.0,1.000000,0.714286,0.714286,0.571429,0.428571,0.428571,0.428571,0.428571,0.285714,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
avg_num_used,NaN,NaN,<NA>,<NA>,2.428571,2.0,2.0,2.571429,1.800000,3.000000,1.000000,1.666667,1.000000,2.000000,1.666667,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000
kb5kFk-l9D4R3-fvbFkf,2023年02月22日(水),埼玉県,80,6,2.000000,2.0,2.0,2.000000,2.000000,3.000000,1.000000,2.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
YDcYac-Q6u7MP-8x8Gc4,2023年02月16日(木),大阪府,48,2,2.000000,2.0,2.0,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XXyyyy-OD33yS-yUMppR,2023年02月16日(木),東京都,70,9,2.000000,2.0,2.0,2.000000,0.000000,3.000000,1.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
DxGac8-F5wrxu-x48Dcc,2023年02月11日(土),埼玉県,64,4,3.000000,2.0,2.0,4.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ngnniN-KoYqjx-QngPgN,2023年02月10日(金),大阪府,100,12,2.000000,2.0,2.0,2.000000,2.000000,3.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FFvvFv-cchC3K-kkf1Fk,2023年02月05日(日),京都,64,7,4.000000,2.0,2.0,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000,1.000000,0.000000
5fFkF5-y8CWzf-vVwvkf,2023年02月05日(日),埼玉県,64,3,2.000000,2.0,2.0,3.000000,2.000000,3.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
df_list[1]

,date,prefecture,num_players,rank,巢穴球,高級球,森林封印石,山谷回音喇叭,洗翠的沉重球,離洞繩,超群眼鏡,講究腰帶,連擊的卷軸 旋渦之卷,對戰VIP參加證,朋友手冊,葉隱披風,捕獲香氛,寶可夢交替,蝴蝶結徽章,寶可裝置3.0
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,1.0,1.000000,0.714286,0.571429,0.571429,0.571429,0.428571,0.428571,0.428571,0.285714,0.285714,0.142857,0.142857,0.142857,0.142857
avg_num_used,NaN,NaN,<NA>,<NA>,3.714286,4.0,2.285714,1.000000,1.000000,1.250000,1.000000,1.333333,1.666667,1.000000,1.000000,1.500000,4.000000,1.000000,1.000000,2.000000
kb5kFk-l9D4R3-fvbFkf,2023年02月22日(水),埼玉県,80,6,4.000000,4.0,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
YDcYac-Q6u7MP-8x8Gc4,2023年02月16日(木),大阪府,48,2,4.000000,4.0,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
XXyyyy-OD33yS-yUMppR,2023年02月16日(木),東京都,70,9,4.000000,4.0,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DxGac8-F5wrxu-x48Dcc,2023年02月11日(土),埼玉県,64,4,4.000000,4.0,2.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
ngnniN-KoYqjx-QngPgN,2023年02月10日(金),大阪府,100,12,4.000000,4.0,3.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FFvvFv-cchC3K-kkf1Fk,2023年02月05日(日),京都,64,7,2.000000,4.0,2.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,4.000000,0.000000,0.000000,0.000000
5fFkF5-y8CWzf-vVwvkf,2023年02月05日(日),埼玉県,64,3,4.000000,4.0,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
df_list[2]

,date,prefecture,num_players,rank,博士的研究,可爾妮的氣勢,老大的指令,裁判,莎莉娜,杜娟,芽米,查克洛,希嘉娜的決心,珠貝,奇巴納,藤樹
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,1.000000,1.000000,0.571429,0.571429,0.571429,0.428571,0.428571,0.428571,0.285714,0.285714,0.142857
avg_num_used,NaN,NaN,<NA>,<NA>,3.142857,1.285714,2.285714,1.500000,1.000000,1.000000,2.333333,1.000000,1.666667,2.000000,1.000000,1.000000
kb5kFk-l9D4R3-fvbFkf,2023年02月22日(水),埼玉県,80,6,4.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000
YDcYac-Q6u7MP-8x8Gc4,2023年02月16日(木),大阪府,48,2,2.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000
XXyyyy-OD33yS-yUMppR,2023年02月16日(木),東京都,70,9,3.000000,2.000000,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000
DxGac8-F5wrxu-x48Dcc,2023年02月11日(土),埼玉県,64,4,2.000000,1.000000,2.000000,2.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ngnniN-KoYqjx-QngPgN,2023年02月10日(金),大阪府,100,12,4.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
FFvvFv-cchC3K-kkf1Fk,2023年02月05日(日),京都,64,7,3.000000,1.000000,2.000000,2.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5fFkF5-y8CWzf-vVwvkf,2023年02月05日(日),埼玉県,64,3,4.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000


In [18]:
df_list[3]

,date,prefecture,num_players,rank,水之塔,通頂雪道
pick_rate,NaN,NaN,<NA>,<NA>,0.857143,0.142857
avg_num_used,NaN,NaN,<NA>,<NA>,2.833333,4.000000
kb5kFk-l9D4R3-fvbFkf,2023年02月22日(水),埼玉県,80,6,3.000000,0.000000
YDcYac-Q6u7MP-8x8Gc4,2023年02月16日(木),大阪府,48,2,2.000000,0.000000
XXyyyy-OD33yS-yUMppR,2023年02月16日(木),東京都,70,9,3.000000,0.000000
DxGac8-F5wrxu-x48Dcc,2023年02月11日(土),埼玉県,64,4,3.000000,0.000000
ngnniN-KoYqjx-QngPgN,2023年02月10日(金),大阪府,100,12,3.000000,0.000000
FFvvFv-cchC3K-kkf1Fk,2023年02月05日(日),京都,64,7,0.000000,4.000000
5fFkF5-y8CWzf-vVwvkf,2023年02月05日(日),埼玉県,64,3,3.000000,0.000000


In [19]:
df_list[4]

,date,prefecture,num_players,rank,連擊能量,螺旋能量,基本雷能量,V防守能量
pick_rate,NaN,NaN,<NA>,<NA>,1.0,1.000000,0.714286,0.142857
avg_num_used,NaN,NaN,<NA>,<NA>,4.0,2.857143,4.600000,1.000000
kb5kFk-l9D4R3-fvbFkf,2023年02月22日(水),埼玉県,80,6,4.0,2.000000,5.000000,0.000000
YDcYac-Q6u7MP-8x8Gc4,2023年02月16日(木),大阪府,48,2,4.0,4.000000,0.000000,1.000000
XXyyyy-OD33yS-yUMppR,2023年02月16日(木),東京都,70,9,4.0,4.000000,3.000000,0.000000
DxGac8-F5wrxu-x48Dcc,2023年02月11日(土),埼玉県,64,4,4.0,3.000000,6.000000,0.000000
ngnniN-KoYqjx-QngPgN,2023年02月10日(金),大阪府,100,12,4.0,2.000000,4.000000,0.000000
FFvvFv-cchC3K-kkf1Fk,2023年02月05日(日),京都,64,7,4.0,3.000000,0.000000,0.000000
5fFkF5-y8CWzf-vVwvkf,2023年02月05日(日),埼玉県,64,3,4.0,2.000000,5.000000,0.000000


In [20]:
len(decks["others"])

85